In [3]:
from langchain.chains.llm import LLMChain
from langchain_community.llms import Tongyi

llm = Tongyi()
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.transform import TransformChain
from langchain.chains.sequential import SimpleSequentialChain



# 第一个任务
def transform_func(inputs:dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n".join(text.split("\n")[:3])
    return {"output_text":shortened_text}

#文档转换链
transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output_text"],
    transform=transform_func
)

# 第二个任务
template = """对下面的文字进行总结:
{output_text}

总结:"""

prompt = PromptTemplate(
    input_variables=["output_text"],
    template=template
)
llm_chain = LLMChain(
    llm = Tongyi(),
    prompt=prompt
)

#使用顺序链连接起来
squential_chain = SimpleSequentialChain(
    chains=[transform_chain,llm_chain],
    verbose=True
)

with open("doc/NBA新闻.txt",encoding='utf-8') as f:
    letters = f.read()
squential_chain.invoke(letters)



> Entering new SimpleSequentialChain chain...
在2024年NBA总决赛的最后一场比赛中，丹佛掘金在主场以94-89击败热火，以总比分4-1成功夺得NBA总冠军。这是球队历史上第一次获得总冠军，也是球队自1976年进入NBA以来的最佳成绩。丹佛掘金队的成功主要归功于他们的领袖球员约基奇的出色表现。
在总决赛的五场比赛中，约基奇展现出了惊人的统治力。他场均贡献30.2分、14个篮板和7.2次助攻，成为球队得分、篮板和助攻的核心。约基奇在进攻端展现出了全面的技术，他的得分能力和篮板能力让热火队无可奈何。同时，他还展现出了出色的组织能力，为球队创造了很多得分机会。
在总决赛的最后一场比赛中，约基奇更是发挥出色。他在关键时刻承担责任，不仅在进攻端贡献了关键得分，还在防守端起到了重要作用。他的领导能力和稳定性为球队赢得了决胜的胜利。
在2024年NBA总决赛中，丹佛掘金以4-1的总比分击败热火，首次夺得NBA总冠军。球队的成功主要得益于领袖球员约基奇在整个系列赛中的出色表现。约基奇在五场比赛中场均得到30.2分、14个篮板和7.2次助攻，成为球队的得分、篮板和助攻核心。他在最后一场比赛中的关键发挥尤其突出，展现了卓越的领导力和稳定性，为球队赢得冠军立下了汗马功劳。

> Finished chain.


{'input': '在2024年NBA总决赛的最后一场比赛中，丹佛掘金在主场以94-89击败热火，以总比分4-1成功夺得NBA总冠军。这是球队历史上第一次获得总冠军，也是球队自1976年进入NBA以来的最佳成绩。丹佛掘金队的成功主要归功于他们的领袖球员约基奇的出色表现。\n在总决赛的五场比赛中，约基奇展现出了惊人的统治力。他场均贡献30.2分、14个篮板和7.2次助攻，成为球队得分、篮板和助攻的核心。约基奇在进攻端展现出了全面的技术，他的得分能力和篮板能力让热火队无可奈何。同时，他还展现出了出色的组织能力，为球队创造了很多得分机会。\n在总决赛的最后一场比赛中，约基奇更是发挥出色。他在关键时刻承担责任，不仅在进攻端贡献了关键得分，还在防守端起到了重要作用。他的领导能力和稳定性为球队赢得了决胜的胜利。\n约基奇荣获总决赛最有价值球员（MVP）毫无悬念。他在总决赛中的出色表现让他成为了不可或缺的球队核心，也让他获得了职业生涯中的首个总冠军。这一荣誉不仅是对他个人努力的认可，也是对他带领球队取得成功的肯定。\n随着约基奇的崛起，丹佛掘金队在过去几个赛季中逐渐崭露头角。他的全面发展和领导能力使他成为了球队的核心和灵魂人物。通过这次总决赛的胜利，约基奇不仅实现了自己的篮球梦想，也为球队带来了无比的荣耀。\n约基奇带领丹佛掘金赢得2024年NBA总冠军，并凭借出色的表现获得总决赛最有价值球员（MVP）的荣誉。他在总决赛期间的统治力和全面能力使他成为球队的核心，同时也展现了他的领导才能。这次胜利不仅是约基奇个人职业生涯的里程碑，也是丹佛掘金队迈向更高荣耀的关键一步。随着约基奇的领导，丹佛掘金队有望在未来继续取得更多的成功。',
 'output': '在2024年NBA总决赛中，丹佛掘金以4-1的总比分击败热火，首次夺得NBA总冠军。球队的成功主要得益于领袖球员约基奇在整个系列赛中的出色表现。约基奇在五场比赛中场均得到30.2分、14个篮板和7.2次助攻，成为球队的得分、篮板和助攻核心。他在最后一场比赛中的关键发挥尤其突出，展现了卓越的领导力和稳定性，为球队赢得冠军立下了汗马功劳。'}

In [16]:
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms.tongyi import Tongyi
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

# 加载问答
# loader = PyPDFLoader("doc/demo.pdf")
loader = TextLoader("doc/NBA新闻.txt",encoding='utf-8')
docs = loader.load()

#split
text_splitter = CharacterTextSplitter(separator="\n",chunk_size=400, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

prompt_template = """对以下文字做简洁的总结:
{text}
简洁的总结:"""

prompt = PromptTemplate.from_template(prompt_template)


refine_template = (
    "你的任务是产生最终摘要\n"
    "我们已经提供了一个到某个特定点的现有回答:{existing_answer}\n"
    "我们有机会通过下面的一些更多上下文来完善现有的回答(仅在需要时使用).\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "根据新的上下文，用中文完善原始回答.\n"
    "如果上下文没有用处，返回原始回答。"
)

refine_prompt = PromptTemplate.from_template(refine_template)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt = refine_prompt,
    return_intermediate_steps=True,
    input_key = "documents",
    output_key = "output_text",
)
result = chain.invoke({"documents":split_docs})
# print(result)
# print(result["output_text"])
print("\n\n".join(result["intermediate_steps"][:2]))

在2024年NBA总决赛中，丹佛掘金以4-1的总比分击败热火，首次夺得总冠军。球队的成功主要得益于领袖球员约基奇在总决赛中的出色表现，他场均贡献30.2分、14个篮板和7.2次助攻，并在最后一场比赛的关键时刻发挥了决定性作用。

在2024年NBA总决赛中，丹佛掘金以4-1的总比分击败了热火，首次夺得了总冠军。这一胜利主要归功于球队领袖尼古拉·约基奇在总决赛中的卓越表现。他场均贡献了30.2分、14个篮板和7.2次助攻，并在关键的第五场比赛中发挥了决定性的作用。约基奇毫无悬念地荣获了总决赛最有价值球员（MVP）的称号。他的全面发展与领导能力不仅巩固了他在球队中的核心地位，也为他赢得了职业生涯的第一个总冠军。这次胜利不仅是对约基奇个人努力的认可，也是对他带领球队取得成功的肯定。随着约基奇的崛起，丹佛掘金在过去几个赛季中逐渐崭露头角，这次的总冠军不仅是约基奇个人职业生涯的重要里程碑，也是丹佛掘金队迈向更高荣耀的关键一步。


In [17]:
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.combine_documents.reduce import ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.tongyi import Tongyi
from langchain.chains.llm import LLMChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

#load pdf
# loader = PyPDFLoader("doc/demo.pdf")
loader = TextLoader("doc/NBA新闻.txt",encoding='utf-8')
docs = loader.load()

text_splitter = CharacterTextSplitter(separator="\n",chunk_size=400, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

#map chain
map_template = """对以下文字做简洁的总结:
"{content}"
简洁的总结:"""
map_prompt = PromptTemplate.from_template(map_template)

# 定义任务
map_chain = LLMChain(
    llm=llm, #llm
    prompt=map_prompt,
)

#reduce chain
reduce_template = """以下是一个摘要集合:
{doc_summaries}
将上述摘要与所有关键细节进行总结.
总结:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# 定义reduce任务
reduce_chain = LLMChain(
    prompt=reduce_prompt,
    llm=llm,
)

# stuff chain
stuff_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_summaries",
)

# reduce chain
reduce_final_chain = ReduceDocumentsChain(
    combine_documents_chain=stuff_chain,
    #超过4000个token就会切入到下一个stuff_chain
    collapse_documents_chain=stuff_chain,
    token_max=4000,
)

# map reduce chain
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    document_variable_name="content",
    reduce_documents_chain=reduce_final_chain,
)

# 默认使用了hugging face上的 GPT-2 模型，下载有问题会报错

summary = map_reduce_chain.invoke(split_docs)
print(summary)

OSError: Can't load tokenizer for 'gpt2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'gpt2' is the correct path to a directory containing all relevant files for a GPT2TokenizerFast tokenizer.

In [21]:
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.combine_documents.reduce import ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.tongyi import Tongyi
from langchain.chains.llm import LLMChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

#load
# loader = PyPDFLoader("doc/demo.pdf")

loader = TextLoader("doc/NBA新闻.txt",encoding='utf-8')
docs = loader.load()
#split
text_splitter = CharacterTextSplitter(separator="\n",chunk_size=200, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

chain = load_qa_with_sources_chain(
    llm, 
    chain_type="map_rerank", 
    metadata_keys=['source'], 
    return_intermediate_steps=True,
)
print(chain)
query = "中文回答这篇文章的主要内容是什么？"
result = chain.invoke({"input_documents":split_docs,"question":query})
result

llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=RegexParser(regex='(.*?)\\nScore: (\\d*)', output_keys=['answer', 'score']), template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nIn addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\nQuestion: [question here]\nHelpful Answer: [answer here]\nScore: [score between 0 and 100]\n\nHow to determine the score:\n- Higher is a better answer\n- Better responds fully to the asked question, with sufficient level of detail\n- If you do not know the answer based on the context, that should be a score of 0\n- Don't be overconfident!\n\nExample #1\n\nContext:\n---------\nApples are red\n---------\nQuestion: what color are apples?\nHelpful Answer: red\nScore: 100\n\nExample #2\n\nContext:\n---------\

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'input_documents': [Document(page_content='在2024年NBA总决赛的最后一场比赛中，丹佛掘金在主场以94-89击败热火，以总比分4-1成功夺得NBA总冠军。这是球队历史上第一次获得总冠军，也是球队自1976年进入NBA以来的最佳成绩。丹佛掘金队的成功主要归功于他们的领袖球员约基奇的出色表现。', metadata={'source': 'doc/NBA新闻.txt'}),
  Document(page_content='在总决赛的五场比赛中，约基奇展现出了惊人的统治力。他场均贡献30.2分、14个篮板和7.2次助攻，成为球队得分、篮板和助攻的核心。约基奇在进攻端展现出了全面的技术，他的得分能力和篮板能力让热火队无可奈何。同时，他还展现出了出色的组织能力，为球队创造了很多得分机会。', metadata={'source': 'doc/NBA新闻.txt'}),
  Document(page_content='在总决赛的最后一场比赛中，约基奇更是发挥出色。他在关键时刻承担责任，不仅在进攻端贡献了关键得分，还在防守端起到了重要作用。他的领导能力和稳定性为球队赢得了决胜的胜利。\n约基奇荣获总决赛最有价值球员（MVP）毫无悬念。他在总决赛中的出色表现让他成为了不可或缺的球队核心，也让他获得了职业生涯中的首个总冠军。这一荣誉不仅是对他个人努力的认可，也是对他带领球队取得成功的肯定。', metadata={'source': 'doc/NBA新闻.txt'}),
  Document(page_content='随着约基奇的崛起，丹佛掘金队在过去几个赛季中逐渐崭露头角。他的全面发展和领导能力使他成为了球队的核心和灵魂人物。通过这次总决赛的胜利，约基奇不仅实现了自己的篮球梦想，也为球队带来了无比的荣耀。', metadata={'source': 'doc/NBA新闻.txt'}),
  Document(page_content='约基奇带领丹佛掘金赢得2024年NBA总冠军，并凭借出色的表现获得总决赛最有价值球员（MVP）的荣誉。他在总决赛期间的统治力和全面能力使他成为球队的核心，同时也展现了他的领导才能。这次胜利不仅是约基奇个人职业生涯的里程碑，也是丹佛掘金队迈向更高荣耀的关键一步。随着约基奇的领导，丹佛掘金队有望在未来